In [ ]:
from utils.load_data import load_images
import torch

train_val_dataset, _, _, _ = load_images(
        "/home/catarina_caldeira/Imagens/SynthRAD2025/synthRAD2025_Task1_Train/Task1", 25
    )
print(train_val_dataset)

Loading dataset:  80%|████████  | 73/91 [00:36<00:09,  1.97it/s]


In [ ]:
from models.generator_monai import generator_unet

best_model_path = "/home/catarina_caldeira/Desktop/code/validations/saved_models/SynthRAD2023/generator/5layers_unet_BN_200_lr_0.01/fold_4/val_bestmodel_fold4_epoch194_interval_0-199.pth"
model = generator_unet([64, 128, 256, 512, 512], [2, 2, 2, 2], "BATCH", 0)

checkpoint = torch.load(best_model_path, weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu') 
model.to(device)

Sequential(
  (0): UNet(
    (model): Sequential(
      (0): Convolution(
        (conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (adn): ADN(
          (N): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (D): Dropout(p=0.0, inplace=False)
          (A): ReLU()
        )
      )
      (1): SkipConnection(
        (submodule): Sequential(
          (0): Convolution(
            (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
            (adn): ADN(
              (N): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (D): Dropout(p=0.0, inplace=False)
              (A): ReLU()
            )
          )
          (1): SkipConnection(
            (submodule): Sequential(
              (0): Convolution(
                (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
                (adn): ADN(
                  (N): Ba

In [ ]:
    # SynthRAD2025
from utils.load_data import load_images, load_hn_centerD_images    
from monai.utils import set_determinism
set_determinism(seed=42)
dataset_path="/home/catarina_caldeira/Imagens/SynthRAD2025/synthRAD2025_Task1_Train/Task1"
_, tests_datasets, _, test_patient_folders = load_images(dataset_path, 25, True)
    
print(test_patient_folders)

Loading dataset: 100%|██████████| 25/25 [00:13<00:00,  1.85it/s]

{('A', 'HN'): ['/home/catarina_caldeira/Imagens/SynthRAD2025/synthRAD2025_Task1_Train/Task1/HN/1HNA139', '/home/catarina_caldeira/Imagens/SynthRAD2025/synthRAD2025_Task1_Train/Task1/HN/1HNA124'], ('C', 'HN'): ['/home/catarina_caldeira/Imagens/SynthRAD2025/synthRAD2025_Task1_Train/Task1/HN/1HNC118', '/home/catarina_caldeira/Imagens/SynthRAD2025/synthRAD2025_Task1_Train/Task1/HN/1HNC107'], ('B', 'AB'): ['/home/catarina_caldeira/Imagens/SynthRAD2025/synthRAD2025_Task1_Train/Task1/AB/1ABB008', '/home/catarina_caldeira/Imagens/SynthRAD2025/synthRAD2025_Task1_Train/Task1/AB/1ABB098'], ('A', 'AB'): ['/home/catarina_caldeira/Imagens/SynthRAD2025/synthRAD2025_Task1_Train/Task1/AB/1ABA109', '/home/catarina_caldeira/Imagens/SynthRAD2025/synthRAD2025_Task1_Train/Task1/AB/1ABA082'], ('C', 'AB'): ['/home/catarina_caldeira/Imagens/SynthRAD2025/synthRAD2025_Task1_Train/Task1/AB/1ABC019'], ('B', 'TH'): ['/home/catarina_caldeira/Imagens/SynthRAD2025/synthRAD2025_Task1_Train/Task1/TH/1THB021', '/home/cat

In [25]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import os 

a_keys = [k for k in tests_datasets if k[1] == "TH"]

# Extract datasets per region
a_data = [tests_datasets[k] for k in a_keys]
    
combined_data = torch.utils.data.ConcatDataset(a_data)
output_dir = "/home/catarina_caldeira/Desktop/code/utils/images_thesis"
for i,patient_data in enumerate(combined_data):
    mri = patient_data['mr'][0].cpu().numpy()  
    ct = patient_data['ct'][0].cpu().numpy()
    mask = patient_data['mask'][0].cpu().numpy()
    
    if i==2:
        patient_id = combined_data[i]

        slice_idx = 22

        # Extract axial slices and rotate 90° clockwise
        mri_slice = np.rot90(mri[:, :,slice_idx],k=-1)
        ct_slice = np.rot90(ct[:, :,slice_idx],k=-1)
        mask_slice = np.rot90(mask[:, :,slice_idx],k=-1)


        # Plot
        fig, axs = plt.subplots(1, 3, figsize=(12, 4))
        axs[0].imshow(mri_slice, cmap="gray")
        axs[1].imshow(ct_slice, cmap="gray")
        axs[2].imshow(mask_slice, cmap="gray")


        for ax in axs:
            ax.axis("off")

        output_path = os.path.join(output_dir, f"{i}_TH_region.png")
        plt.tight_layout()
        plt.savefig(output_path, bbox_inches="tight", dpi=300)
        plt.close()


In [10]:
import numpy as np
from torch.utils.data import Subset
from sklearn.model_selection import StratifiedKFold
from monai.transforms import (
    Compose,
    ScaleIntensityd,
    MaskIntensityd,
    ScaleIntensityRanged,
    ToTensord,
    RandSpatialCropd,
    SpatialPadd,
    Lambdad,
)
from torchmetrics.regression import MeanAbsoluteError
from torchmetrics.image import PeakSignalNoiseRatio
from torchmetrics.image import StructuralSimilarityIndexMeasure,MultiScaleStructuralSimilarityIndexMeasure

from monai.data import DataLoader,Dataset
from monai.utils import set_determinism
from monai.inferers import SlidingWindowInferer
import os
from models.generator_monai import generator_unet
from utils.load_data import load_images
import torch

set_determinism(seed=42)

train_val_dataset, _, train_val_patient_folders, _ = load_images(
        "/home/catarina_caldeira/Imagens/SynthRAD2023dataset/Task1/pelvis", 23
    )
print(train_val_dataset)

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu') 

combined_data = torch.utils.data.ConcatDataset(list(train_val_dataset.values())) # pelvis data from center A and C
all_patients_path = torch.utils.data.ConcatDataset(list(train_val_patient_folders.values())) 
    
    
a_keys = list(filter(lambda k: k[0] == "A", train_val_dataset))
c_keys = list(filter(lambda k: k[0] == "C", train_val_dataset))

y_centers = np.concatenate([
        np.zeros(sum(len(train_val_dataset[k]) for k in a_keys)),
        np.ones(sum(len(train_val_dataset[k]) for k in c_keys)),
    ])


mae = MeanAbsoluteError().to(device)
psnr = PeakSignalNoiseRatio().to(device)
ssim = StructuralSimilarityIndexMeasure().to(device)
ms_ssim = MultiScaleStructuralSimilarityIndexMeasure().to(device)
    
    
mae_list,psnr_list, ssim_list, msssim_list = [],[], [], []
# Set up k-fold cross-validation
splits = 5
    
# Set up Stratified k-fold cross-validation
skf  = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)
for fold, (_, val_index) in enumerate(skf.split(combined_data, y_centers)):

        # Split the dataset
        valid_set = Subset(combined_data, val_index)
        val_patients_path = [all_patients_path[i] for i in val_index.tolist()]
        print(f"Validation patients: {val_patients_path}")


        transforms_val = Compose(
            [
                #SpatialPadd(keys=["mr", "ct", "mask"], spatial_size=(256, 256, -1)),
                MaskIntensityd(keys=["mr", "ct"], mask_key="mask"),
                ToTensord(keys=["mr", "ct"]),
            ]
        )
        
        monai_valid_dataset = Dataset(
            data=valid_set, transform=transforms_val
        )  
        
        valid_loader = DataLoader(monai_valid_dataset, batch_size=1, num_workers=8, persistent_workers=True,
            pin_memory=True)
        
        best_model_path = "/home/catarina_caldeira/Desktop/code/validations/saved_models/SynthRAD2023/generator/5layers_unet_BN_200_lr_0.00001"
        model = generator_unet([64, 128, 256, 512, 512], [2, 2, 2, 2], "BATCH", 0)

        
        fold_model_path = os.path.join(best_model_path, f"fold_{fold}")
        #print(fold_model_path)
        
        model_file = None
        if os.path.exists(fold_model_path):
            for file in os.listdir(fold_model_path):
                if file.startswith("val_bestmodel_fold") and file.endswith("199.pth"):
                    model_file = os.path.join(fold_model_path, file)
                    print(f"model file: {model_file}")
        
        if model_file:
            checkpoint = torch.load(model_file, weights_only=False)
            model.load_state_dict(checkpoint['model_state_dict'])
            model.to(device)
            
            model.eval()
            
            with torch.no_grad():
                for batch in valid_loader:
                    real_A = batch["mr"].to(device)  # Shape: [B, C, H, W, D], C=1
                    real_B = batch["ct"].to(device)  # Shape: [B, C, H, W, D]
                    mask = batch["mask"].to(device)
                    
                    real_A = torch.swapaxes(real_A, -1, 0)  # Shape: [D, 1, H, W, 1]
                    real_B = torch.swapaxes(real_B, -1, 0)
                    mask = torch.swapaxes(mask, -1, 0)


                    real_A = real_A.squeeze(
                                -1
                    )  # Shape: [D, 1, H, W] -> n(=D) images 2D
                    real_B = real_B.squeeze(-1)
                    mask = mask.squeeze(-1)

                    
                    
                    inferer = SlidingWindowInferer(
                                    roi_size=(256, 256),
                                    mode="constant",
                                    progress=True,
                                    overlap=0.0,
                                )
                
                
                    fake_B = inferer(real_A, model)
                    fake_B_mask = fake_B * mask.float()  # masked prediction
                    #mae = torch.mean((fake_B_mask-real_B)**2)
                    mask_bool = (
                    mask == 1
                    )
                    mae_per_batch = mae(
                        fake_B_mask[mask_bool], real_B[mask_bool]
                    ).item()
                    psnr_per_batch = psnr(fake_B_mask, real_B).item()
                    ssim_per_batch = ssim(fake_B_mask, real_B).item()
                    ms_ssim_per_batch = ms_ssim(fake_B_mask, real_B).item()

                    print(f"PSNR: {psnr_per_batch}")
                    print(f"MAE PER BATCH: {mae_per_batch}")
                    print(f"SSIM: {ssim_per_batch}")
                    print(f"MS-SSIM: {ms_ssim_per_batch}")
                    mae_list.append(mae_per_batch)
                    psnr_list.append(psnr_per_batch)
                    ssim_list.append(ssim_per_batch)
                    msssim_list.append(ms_ssim_per_batch)
                    

            
            
            
            
            
    

Loading dataset:   0%|          | 0/120 [00:00<?, ?it/s]

Loading dataset: 100%|██████████| 60/60 [00:14<00:00,  4.11it/s]


{('A', 'P'): <torch.utils.data.dataset.Subset object at 0x7e078670b050>, ('C', 'P'): <torch.utils.data.dataset.Subset object at 0x7e0786517a10>}
Validation patients: ['/home/catarina_caldeira/Imagens/SynthRAD2023dataset/Task1/pelvis/1PA168', '/home/catarina_caldeira/Imagens/SynthRAD2023dataset/Task1/pelvis/1PA165', '/home/catarina_caldeira/Imagens/SynthRAD2023dataset/Task1/pelvis/1PA038', '/home/catarina_caldeira/Imagens/SynthRAD2023dataset/Task1/pelvis/1PA010', '/home/catarina_caldeira/Imagens/SynthRAD2023dataset/Task1/pelvis/1PA031', '/home/catarina_caldeira/Imagens/SynthRAD2023dataset/Task1/pelvis/1PA105', '/home/catarina_caldeira/Imagens/SynthRAD2023dataset/Task1/pelvis/1PA178', '/home/catarina_caldeira/Imagens/SynthRAD2023dataset/Task1/pelvis/1PA159', '/home/catarina_caldeira/Imagens/SynthRAD2023dataset/Task1/pelvis/1PA040', '/home/catarina_caldeira/Imagens/SynthRAD2023dataset/Task1/pelvis/1PA164', '/home/catarina_caldeira/Imagens/SynthRAD2023dataset/Task1/pelvis/1PA088', '/home/c

100%|██████████| 774/774 [00:01<00:00, 762.74it/s]


PSNR: 19.109451293945312
MAE PER BATCH: 0.18847130239009857
SSIM: 0.5531085133552551
MS-SSIM: 0.5914263129234314


...

PSNR: 22.88092803955078
MAE PER BATCH: 0.11483144760131836
SSIM: 0.5722323060035706
MS-SSIM: 0.6490359902381897


100%|██████████| 344/344 [00:00<00:00, 731.87it/s]


PSNR: 22.84407615661621
MAE PER BATCH: 0.11392444372177124
SSIM: 0.626372218132019
MS-SSIM: 0.6704877614974976


100%|██████████| 400/400 [00:00<00:00, 677.64it/s]


PSNR: 23.288799285888672
MAE PER BATCH: 0.11022763699293137
SSIM: 0.5739891529083252
MS-SSIM: 0.6416633725166321


In [8]:
torch.sort(torch.as_tensor(mae_list))
torch.sort(torch.as_tensor(psnr_list))
torch.sort(torch.as_tensor(ssim_list))
torch.sort(torch.as_tensor(msssim_list))


torch.return_types.sort(
values=tensor([0.6728, 0.7102, 0.7236, 0.7393, 0.7412, 0.7488, 0.7492, 0.7599, 0.7607,
        0.7633, 0.7646, 0.7654, 0.7665, 0.7668, 0.7678, 0.7679, 0.7685, 0.7702,
        0.7703, 0.7733, 0.7735, 0.7749, 0.7752, 0.7767, 0.7774, 0.7779, 0.7783,
        0.7784, 0.7799, 0.7799, 0.7802, 0.7809, 0.7815, 0.7822, 0.7824, 0.7830,
        0.7837, 0.7837, 0.7838, 0.7841, 0.7844, 0.7855, 0.7856, 0.7863, 0.7868,
        0.7884, 0.7887, 0.7899, 0.7908, 0.7910, 0.7913, 0.7923, 0.7929, 0.7933,
        0.7942, 0.7958, 0.7962, 0.7963, 0.7971, 0.7979, 0.7979, 0.7981, 0.7985,
        0.7994, 0.8001, 0.8007, 0.8008, 0.8009, 0.8016, 0.8023, 0.8032, 0.8037,
        0.8039, 0.8040, 0.8042, 0.8042, 0.8045, 0.8065, 0.8074, 0.8077, 0.8098,
        0.8118, 0.8132, 0.8138, 0.8142, 0.8144, 0.8144, 0.8145, 0.8146, 0.8151,
        0.8153, 0.8162, 0.8168, 0.8169, 0.8182, 0.8196, 0.8202, 0.8203, 0.8207,
        0.8207, 0.8209, 0.8211, 0.8217, 0.8221, 0.8223, 0.8229, 0.8244, 0.8253,
        

In [ ]:
has_nan_3 = torch.isnan(torch.as_tensor(msssim_list)).any()
has_nan_2 = torch.isnan(torch.as_tensor(ssim_list)).any()
has_nan_1 = torch.isnan(torch.as_tensor(psnr_list)).any()

has_nan = torch.isnan(torch.as_tensor(mae_list)).any()
print(f"Contains nan values psnr: {has_nan.item()}")
print(f"Contains nan values mae: {has_nan_1.item()}")
print(f"Contains nan values ssim: {has_nan_2.item()}")
print(f"Contains nan values ms-ssim: {has_nan_3.item()}")


Contains nan values psnr: False
Contains nan values mae: False
Contains nan values ssim: False
Contains nan values ms-ssim: False
